In [ ]:
!pip install wldhx.yadisk-direct datasets evaluate sentence-transformers rank_bm25
!pip install --upgrade transformers accelerate

!curl -L $(yadisk-direct https://disk.yandex.ru/d/HpBl9kxYSaVSqw) -o data.zip
!unzip data.zip >> /dev/null

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, losses, models, util, InputExample
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import logging
import os
from tqdm.auto import tqdm
import sys
import time
from rank_bm25 import BM25Okapi
from typing import List, Dict, Union

# Enhanced logging setup
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
logger = logging.getLogger(__name__)

# Load all data with proper types
documents_df = pd.read_csv('kaggle_3_documents.csv', dtype={'id': str})
train_df = pd.read_csv('kaggle_3_train.csv', dtype={'id': str})
test_df = pd.read_csv('kaggle_3_test.csv', dtype={'id': str})

# Keep original sample for quick testing
train_df_sample = train_df[0:500].copy()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
class EnhancedRetriever:
    """
    Complete improved retriever with all original functionality plus:
    - Better model (bge-base)
    - Hard negative sampling
    - BM25 for retrieval augmentation
    - Improved memory efficiency
    - Better evaluation metrics
    """

    def __init__(self, model_name: str = "BAAI/bge-base-en-v1.5"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"Initializing retriever on {self.device} with {model_name}")

        self.model = SentenceTransformer(model_name, device=self.device)

        # Original internal caches
        self.corpus = {}           # dict of {doc_id -> passage}
        self.corpus_ids = []       # list of doc_ids
        self.corpus_texts = []     # list of passages
        self.corpus_embeddings = None

        # Original training/evaluation placeholders
        self.train_examples = []
        self.queries = {}          # {query_id: query_text}
        self.relevant_docs = {}    # {query_id: {doc_id: 1}}

        # New additions
        self.bm25 = None           # For hard negative sampling
        self.batch_size = 128 if 'cuda' in self.device else 32

    def load_corpus(self, documents_df: pd.DataFrame):
        """Original functionality with added preprocessing"""
        logger.info("Loading corpus documents with preprocessing...")

        # Original processing
        documents_df = documents_df.copy()
        documents_df['passage'] = documents_df['passage'].fillna("").astype(str)

        # Add simple preprocessing
        documents_df['passage'] = (
            documents_df['passage']
            .str.lower()
            .str.replace(r'\s+', ' ', regex=True)
            .str.strip()
        )

        # Keep original storage
        self.corpus = dict(zip(documents_df['id'].astype(str), documents_df['passage']))
        self.corpus_ids = list(self.corpus.keys())
        self.corpus_texts = list(self.corpus.values())

        # Initialize BM25 for hard negatives
        tokenized_corpus = [doc.split() for doc in self.corpus_texts]
        self.bm25 = BM25Okapi(tokenized_corpus)

        logger.info(f"Corpus size: {len(self.corpus)} documents")

    def prepare_data(self, train_df: pd.DataFrame,
                   negative_samples: int = 3,
                   eval_ratio: float = 0.2,
                   hard_negatives_ratio: float = 0.3,
                   random_state: int = 42):
        """Fixed version with proper variable scoping"""
        if not self.corpus:
            raise ValueError("No corpus loaded. Call load_corpus(documents_df) first.")

        # Original processing
        train_df = train_df.copy()
        train_df['question'] = train_df['question'].fillna("").astype(str)

        # Determine eval set (original random split)
        total_len = len(train_df)
        eval_size = int(eval_ratio * total_len)
        eval_indices = set(np.random.choice(range(total_len), size=eval_size, replace=False))

        train_examples = []
        queries = {}
        relevant_docs = {}

        # For each row in train data
        for idx, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Preparing data"):
            qid = str(row['id'])
            question_text = row['question']

            # Original relevant IDs parsing
            rel_ids = [
                x.strip() for x in row['relevant_passage_ids'].strip('[]').split(',')
                if x.strip() in self.corpus
            ]

            # 1. Original positive examples
            for rid in rel_ids:
                train_examples.append(InputExample(
                    texts=[question_text, self.corpus[rid]],
                    label=1.0
                ))

            # 2. Enhanced negative sampling
            if negative_samples > 0:
                # Get all irrelevant IDs
                all_irrelevant_ids = [pid for pid in self.corpus.keys() if pid not in rel_ids]

                # Initialize hard_neg_ids as empty list
                hard_neg_ids = []

                # a) Hard negatives using BM25
                num_hard_neg = int(negative_samples * hard_negatives_ratio)
                if num_hard_neg > 0 and len(all_irrelevant_ids) > 0 and self.bm25 is not None:
                    tokenized_query = question_text.split()
                    doc_scores = self.bm25.get_scores(tokenized_query)
                    hard_neg_indices = np.argpartition(doc_scores, -num_hard_neg)[-num_hard_neg:]
                    hard_neg_ids = [self.corpus_ids[i] for i in hard_neg_indices
                                  if self.corpus_ids[i] in all_irrelevant_ids][:num_hard_neg]

                    for nid in hard_neg_ids:
                        train_examples.append(InputExample(
                            texts=[question_text, self.corpus[nid]],
                            label=0.0
                        ))

                # b) Original random negatives
                num_random_neg = negative_samples - num_hard_neg
                if num_random_neg > 0 and len(all_irrelevant_ids) > 0:
                    remaining_ids = [pid for pid in all_irrelevant_ids if pid not in hard_neg_ids]
                    if len(remaining_ids) > 0:  # Only sample if there are IDs remaining
                        random_neg_ids = np.random.choice(
                            remaining_ids,
                            min(num_random_neg, len(remaining_ids)),
                            replace=False
                        )
                        for nid in random_neg_ids:
                            train_examples.append(InputExample(
                                texts=[question_text, self.corpus[nid]],
                                label=0.0
                            ))

            # Original eval set preparation
            if idx in eval_indices:
                queries[qid] = question_text
                relevant_docs[qid] = {doc_id: 1 for doc_id in rel_ids}

        # Store results
        self.train_examples = train_examples
        self.queries = queries
        self.relevant_docs = relevant_docs

        logger.info(f"Total training examples: {len(self.train_examples)}")
        logger.info(f"Eval queries: {len(self.queries)}; corpus size: {len(self.corpus)}")

    # [Rest of the class methods remain exactly the same as before]

    ###########################################################################
    #                  3. TRAINING THE MODEL                                  #
    ###########################################################################

    def train(self, epochs: int = 1,
             evaluation_steps: int = 250,
             warmup_steps: int = 200,
             output_path: str = "output/biomedical-retrieval-model"):
        """Original training with better defaults"""
        if not self.train_examples:
            raise ValueError("No training examples found. Run prepare_data(...) first.")

        logger.info(f"Training model on {len(self.train_examples)} examples...")

        # Original dataloader
        train_dataloader = DataLoader(self.train_examples, batch_size=16, shuffle=True)

        # Original loss
        train_loss = losses.MultipleNegativesRankingLoss(self.model)

        # Original evaluator setup
        if len(self.queries) > 0:
            ir_evaluator = InformationRetrievalEvaluator(
                queries={qid: str(q) for qid, q in self.queries.items()},
                corpus={doc_id: str(doc) for doc_id, doc in self.corpus.items()},
                relevant_docs=self.relevant_docs,
                show_progress_bar=True
            )
        else:
            ir_evaluator = None

        # Enhanced training with better defaults
        self.model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            evaluator=ir_evaluator,
            epochs=epochs,
            evaluation_steps=evaluation_steps if ir_evaluator else 0,
            warmup_steps=warmup_steps,
            output_path=output_path,
            optimizer_params={'lr': 2e-5},  # Better learning rate
            use_amp=True  # Mixed precision
        )
        logger.info("Training complete!")

    ###########################################################################
    #                  4. ONE-STOP CONVENIENCE METHOD: FIT                    #
    ###########################################################################

    def fit(self, train_df: pd.DataFrame,
           negative_samples: int = 3,
           eval_ratio: float = 0.2,
           epochs: int = 1,
           evaluation_steps: int = 250,
           warmup_steps: int = 200,
           output_path: str = "output/biomedical-retrieval-model"):
        """Original convenience method"""
        self.prepare_data(train_df, negative_samples=negative_samples, eval_ratio=eval_ratio)
        self.train(epochs=epochs, evaluation_steps=evaluation_steps,
                  warmup_steps=warmup_steps, output_path=output_path)

    ###########################################################################
    #                  5. PRECOMPUTE CORPUS EMBEDDINGS                        #
    ###########################################################################

    def precompute_corpus_embeddings(self, batch_size: int = None):
        """Original functionality with better batching"""
        if not self.corpus_texts:
            raise ValueError("No corpus found. Did you run load_corpus(...) first?")

        batch_size = batch_size or self.batch_size
        logger.info(f"Computing embeddings for {len(self.corpus_texts)} passages...")

        # Original embedding computation with progress bar
        self.corpus_embeddings = self.model.encode(
            self.corpus_texts,
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_tensor=True,
            device=self.device
        )
        logger.info(f"Corpus embeddings shape: {self.corpus_embeddings.shape}")

    ###########################################################################
    #                  6. RETRIEVAL METHODS (TOP-K)                           #
    ###########################################################################

    def retrieve_top_k(self, query: str, top_k: int = 5):
        """Original retrieval with better batching"""
        if self.corpus_embeddings is None:
            self.precompute_corpus_embeddings()

        # Original query processing
        query_embedding = self.model.encode(
            str(query),
            convert_to_tensor=True,
            device=self.device
        )

        # Original scoring
        cos_scores = util.cos_sim(query_embedding, self.corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=min(top_k, len(self.corpus_texts)))

        # Original result formatting
        results = []
        for score, idx in zip(top_results[0], top_results[1]):
            cid = self.corpus_ids[idx]
            results.append({
                'corpus_id': cid,
                'passage': self.corpus[cid],
                'score': score.item()
            })
        return results

    def retrieve_for_test(self, test_df: pd.DataFrame, top_k: int = 5):
        """Original batch retrieval with better efficiency"""
        if self.corpus_embeddings is None:
            self.precompute_corpus_embeddings()

        # Original processing
        queries = test_df['question'].astype(str).tolist()

        # Process in batches
        all_top_k_ids = []
        for i in range(0, len(queries), self.batch_size):
            batch = queries[i:i+self.batch_size]

            # Original encoding
            query_embeddings = self.model.encode(
                batch,
                convert_to_tensor=True,
                device=self.device
            )

            # Original scoring
            cos_scores = util.cos_sim(query_embeddings, self.corpus_embeddings)

            # Get top-k for each query
            for j in range(len(batch)):
                top_indices = torch.topk(cos_scores[j], k=min(top_k, len(self.corpus_texts))).indices
                all_top_k_ids.append([self.corpus_ids[idx] for idx in top_indices])

        # Original result formatting
        result_df = test_df.copy()
        result_df['relevant_passage_ids'] = all_top_k_ids
        return result_df

    ###########################################################################
    #                  7. EVALUATION / METRIC CALCULATION                     #
    ###########################################################################

    def evaluate(self, test_df: pd.DataFrame, top_k: int = 5, metrics_k: int = 10):
        """Original evaluation with more metrics"""
        retrieved_df = self.retrieve_for_test(test_df, top_k=top_k)

        # Original results preparation
        results_dict = {}
        relevant_dict = {}

        for idx, row in retrieved_df.iterrows():
            qid = str(row['id'])
            retrieved_ids = row['relevant_passage_ids']
            results_dict[qid] = retrieved_ids

            # Original ground truth parsing
            true_ids = [
                x.strip() for x in str(test_df.loc[idx, 'relevant_passage_ids']).strip('[]').split(',')
                if x.strip()
            ]
            relevant_dict[qid] = {doc_id: 1 for doc_id in true_ids if doc_id}

        # Original metrics calculation
        return self.evaluate_ir_metrics(results_dict, relevant_dict, k=metrics_k)

    def evaluate_ir_metrics(self, results: Dict, relevant_docs: Dict, k: int = 10):
        """Original metrics with additional calculations"""
        recall = self._calculate_recall_at_k(results, relevant_docs, k)
        precision = self._calculate_precision_at_k(results, relevant_docs, k)
        mrr = self._calculate_mrr(results, relevant_docs)

        # Additional metrics
        map_score = self._calculate_map(results, relevant_docs, k)
        ndcg = self._calculate_ndcg(results, relevant_docs, k)

        return {
            f"recall@{k}": recall,
            f"precision@{k}": precision,
            "mrr": mrr,
            f"map@{k}": map_score,
            f"ndcg@{k}": ndcg
        }

    # Original metric calculation methods
    def _calculate_recall_at_k(self, results, relevant_docs, k=10):
        recalls = []
        for query_id, retrieved_docs in results.items():
            if query_id in relevant_docs:
                relevant = set(relevant_docs[query_id].keys())
                retrieved = set(retrieved_docs[:k])
                if len(relevant) > 0:
                    recall = len(relevant.intersection(retrieved)) / len(relevant)
                    recalls.append(recall)
        return sum(recalls) / len(recalls) if recalls else 0.0

    def _calculate_precision_at_k(self, results, relevant_docs, k=10):
        precisions = []
        for query_id, retrieved_docs in results.items():
            if query_id in relevant_docs:
                relevant = set(relevant_docs[query_id].keys())
                retrieved = set(retrieved_docs[:k])
                precision = len(relevant.intersection(retrieved)) / len(retrieved) if retrieved else 0.0
                precisions.append(precision)
        return sum(precisions) / len(precisions) if precisions else 0.0

    def _calculate_mrr(self, results, relevant_docs):
        rr_scores = []
        for query_id, retrieved_docs in results.items():
            if query_id in relevant_docs:
                relevant = set(relevant_docs[query_id].keys())
                for rank, doc_id in enumerate(retrieved_docs, 1):
                    if doc_id in relevant:
                        rr_scores.append(1.0 / rank)
                        break
                else:
                    rr_scores.append(0.0)
        return sum(rr_scores) / len(rr_scores) if rr_scores else 0.0

    # New metric calculation methods
    def _calculate_map(self, results, relevant_docs, k=10):
        ap_scores = []
        for query_id, retrieved_docs in results.items():
            if query_id in relevant_docs:
                relevant = set(relevant_docs[query_id].keys())
                if not relevant:
                    continue

                precisions = []
                num_relevant = 0

                for rank, doc_id in enumerate(retrieved_docs[:k], 1):
                    if doc_id in relevant:
                        num_relevant += 1
                        precisions.append(num_relevant / rank)

                if precisions:
                    ap_scores.append(sum(precisions) / min(len(relevant), k))
                else:
                    ap_scores.append(0.0)

        return sum(ap_scores) / len(ap_scores) if ap_scores else 0.0

    def _calculate_ndcg(self, results, relevant_docs, k=10):
        ndcg_scores = []
        for query_id, retrieved_docs in results.items():
            if query_id in relevant_docs:
                relevant = set(relevant_docs[query_id].keys())
                if not relevant:
                    continue

                dcg = 0.0
                for rank, doc_id in enumerate(retrieved_docs[:k], 1):
                    if doc_id in relevant:
                        dcg += 1.0 / np.log2(rank + 1)

                idcg = sum(1.0 / np.log2(i + 1) for i in range(1, min(len(relevant), k) + 1))
                ndcg = dcg / idcg if idcg > 0 else 0.0
                ndcg_scores.append(ndcg)

        return sum(ndcg_scores) / len(ndcg_scores) if ndcg_scores else 0.0

In [ ]:
# Initialize retriever with better model but same interface
retriever = EnhancedRetriever(model_name="BAAI/bge-base-en-v1.5")

# Original corpus loading
retriever.load_corpus(documents_df)

# Quick test before training (original functionality)
result_df = retriever.retrieve_for_test(test_df[0:5], top_k=5)
logger.info('First results before training:')
display(result_df.head())

# Original evaluation before training
metrics = retriever.evaluate(train_df, top_k=10, metrics_k=10)
logger.info('Pre-training metrics:')
display(metrics)

# Train with original parameters but better model
retriever.fit(
    train_df,
    negative_samples=3,
    eval_ratio=0.2,
    epochs=1,
    evaluation_steps=100,
    warmup_steps=50,
    output_path="output/my-retrieval-model"
)

# Original embedding computation
retriever.precompute_corpus_embeddings()

# Original evaluation after training
metrics = retriever.evaluate(train_df, top_k=10, metrics_k=10)
logger.info('Post-training metrics:')
display(metrics)

# Retrieve for test set (original functionality)
result_df = retriever.retrieve_for_test(test_df, top_k=10)
result_df.to_csv('stage_1_prediction.csv', index=None)

logger.info('Saved relevant passage ids to stage_1_prediction.csv')

# Display sample documents and training data
display(documents_df.head())
display(train_df.head())

INFO:__main__:Initializing retriever on cuda with BAAI/bge-base-en-v1.5


2025-04-28 15:50:47,340 - __main__ - INFO - Initializing retriever on cuda with BAAI/bge-base-en-v1.5


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


2025-04-28 15:50:47,345 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:__main__:Saved relevant passage ids to stage_1_prediction.csv


2025-04-28 17:48:45,243 - __main__ - INFO - Saved relevant passage ids to stage_1_prediction.csv


,id,passage
0,9797,New data on viruses isolated from patients wit...
1,11906,We describe an improved method for detecting d...
2,16083,We have studied the effects of curare on respo...
3,23188,Kinetic and electrophoretic properties of 230-...
4,23469,Male Wistar specific-pathogen-free rats aged 2...


,id,question,answer,relevant_passage_ids
0,164,What is the enzymatic activity of the breast c...,E3-ubiquitin ligase activity is the only known...,"[20681793, 22034435, 19088202, 16479151, 24278..."
1,1816,What is the effect of CPEB3 binding to the CPE...,The cytoplasmic polyadenylation element (CPE) ...,"[17923234, 17481902, 25066254, 23776146, 26398..."
2,4363,Which disease do pathogenic NR2F1 variants cause?,Bosch-Boonstra-Schaaf optic atrophy syndrome (...,[26986877]
3,753,Which enzyme is deficient in Gaucher's disease?,Gaucher's disease is caused by deficient lysos...,"[20946052, 18627336, 16781064, 22843412, 24485..."
4,2314,What does davunetide do to microtubules?,Davunetide or NAP is a microtubule-stabilizer.,[24210139]


In [ ]:
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    pipeline,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
import evaluate
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

class Generator:
    """
    A class for training, evaluating, and predicting with a text generation (causal LM) model.

    The class includes methods to:
      1. Build a document dictionary from a DataFrame.
      2. Prepare training data by combining questions, contexts (from relevant passages),
         and answers into prompts.
      3. Tokenize the prompts for training.
      4. Fine-tune the generator using Hugging Face's Trainer.
      5. Evaluate the generator on a sample evaluation set.
      6. Predict answers for a test DataFrame that includes predicted relevant passage IDs.
         In both evaluation and prediction, the prompt is truncated if necessary so that its
         token count plus requested new tokens do not exceed the model's maximum allowed tokens.
    """

    def __init__(self, model_name="gpt2"):
        logger.info(f"Loading model and tokenizer for {model_name}")
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        self.model.config.pad_token_id = self.tokenizer.eos_token_id

        # Placeholders for training data and document dictionary.
        self.doc_dict = {}
        self.qa_df = None
        self.tokenized_dataset = None

    def build_doc_dict(self, documents_df):
        logger.info("Building document dictionary from DataFrame...")
        documents_df["passage"] = documents_df["passage"].fillna("").astype(str)
        self.doc_dict = dict(zip(documents_df["id"].astype(str), documents_df["passage"]))
        logger.info(f"Built doc_dict with {len(self.doc_dict)} documents.")

    def create_prompt(self, question, context, answer=None):
        """Build a prompt string without any truncation."""
        if answer:
            prompt = f"Question: {question}\nContext: {context}\nAnswer: {answer}"
        else:
            prompt = f"Question: {question}\nContext: {context}\nAnswer:"
        return prompt

    def build_truncated_prompt(self, question, context, max_new_tokens):
        """
        Build a prompt while ensuring that its tokenized version does not exceed
        (model's maximum positions - max_new_tokens) tokens. If it does, truncate the context.
        """
        max_positions = self.model.config.n_positions
        # The allowed prompt length is the total positions minus the room we want for new tokens.
        allowed_prompt_length = max_positions - max_new_tokens

        # Build full prompt.
        full_prompt = self.create_prompt(question, context)
        # Tokenize the full prompt.
        inputs = self.tokenizer(full_prompt, return_tensors="pt")
        prompt_length = inputs.input_ids.shape[1]
        if prompt_length <= allowed_prompt_length:
            return full_prompt
        else:
            # Preserve question and "Answer:" markers.
            q_part = f"Question: {question}\nContext: "
            a_part = "\nAnswer:"
            q_tokens = self.tokenizer(q_part, add_special_tokens=False).input_ids
            a_tokens = self.tokenizer(a_part, add_special_tokens=False).input_ids
            fixed_length = len(q_tokens) + len(a_tokens)
            allowed_for_context = allowed_prompt_length - fixed_length
            # Tokenize the full context.
            context_tokens = self.tokenizer(context, add_special_tokens=False).input_ids
            truncated_context_tokens = context_tokens[:max(allowed_for_context, 0)]
            truncated_context = self.tokenizer.decode(truncated_context_tokens, skip_special_tokens=True)
            truncated_prompt = f"Question: {question}\nContext: {truncated_context}\nAnswer:"
            logger.warning(
                f"Prompt truncated: original length {prompt_length}, "
                f"allowed prompt length {allowed_prompt_length}."
            )
            return truncated_prompt

    def prepare_training_data(self, train_df):
        """
        Prepare training data by combining questions, context from relevant passages, and answers into prompts.

        Expects train_df to have the following columns:
          - 'question'
          - 'answer'
          - 'relevant_passage_ids' : a string representation of a list (e.g., "[2, 5]")

        This method builds an internal DataFrame (self.qa_df) with columns:
          - "question", "relevant_docs" (concatenated passages), "answer", "prompt"

        Args:
            train_df (pd.DataFrame): DataFrame containing training samples.
        """
        if not self.doc_dict:
            raise ValueError("Document dictionary is empty. Call build_doc_dict(documents_df) first.")

        logger.info("Preparing training data from DataFrame...")
        records = []
        for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Preparing training data"):
            question = row["question"]
            answer = row["answer"]
            # Parse the relevant_passage_ids string to a list.
            pid_str = str(row["relevant_passage_ids"]).strip("[]")
            doc_ids = [pid.strip() for pid in pid_str.split(",") if pid.strip()]

            # Gather relevant passages based on the doc_dict
            relevant_texts = [self.doc_dict[pid] for pid in doc_ids if pid in self.doc_dict]
            combined_passages = " ".join(relevant_texts)

            prompt = self.create_prompt(question, combined_passages, answer)

            records.append({
                "question": question,
                "relevant_docs": combined_passages,
                "answer": answer,
                "prompt": prompt
            })

        self.qa_df = pd.DataFrame(records)
        logger.info(f"Prepared {len(self.qa_df)} training records.")

    def tokenize_training_data(self, max_length=128):
        """
        Convert the prepared training DataFrame into a tokenized Hugging Face Dataset.

        Args:
            max_length (int): Maximum token length for each prompt.
        """
        if self.qa_df is None:
            raise ValueError("Training data not prepared. Call prepare_training_data(train_df) first.")

        logger.info("Converting training data to Dataset and tokenizing...")
        dataset = Dataset.from_pandas(self.qa_df[["prompt"]])

        def tokenize_function(examples):
            return self.tokenizer(
                examples["prompt"],
                padding="max_length",
                truncation=True,
                max_length=max_length
            )

        self.tokenized_dataset = dataset.map(tokenize_function, batched=True)
        logger.info("Tokenization complete.")

    def train_model(self, output_dir="my_gpt2_generator", num_train_epochs=1, batch_size=8, logging_steps=50):
        """
        Train (fine-tune) the generator model using the tokenized dataset.

        Args:
            output_dir (str): Directory in which to save the fine-tuned model.
            num_train_epochs (int): Number of training epochs.
            batch_size (int): Batch size per device.
            logging_steps (int): Frequency (in steps) at which to log training metrics.
        """
        if self.tokenized_dataset is None:
            raise ValueError("Tokenized training data not found. Run tokenize_training_data(max_length) first.")

        logger.info("Setting up training...")
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False  # GPT-2 uses a causal LM objective, not masked LM
        )

        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=batch_size,
            logging_steps=logging_steps,
            save_total_limit=2,  # adjust as needed
            logging_dir=f"{output_dir}/logs",
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.tokenized_dataset,
            tokenizer=self.tokenizer,
            data_collator=data_collator
        )

        logger.info("Starting training...")
        trainer.train()
        logger.info("Training complete!")

        # Save the model after training
        trainer.save_model(output_dir)
        logger.info(f"Model saved to {output_dir}")

    def evaluate_bert(self, eval_df, max_eval=100, max_new_tokens=32, do_sample=False):
        """
        Evaluate the generator model on a subset of examples using BERTScore.

        This method uses a text generation pipeline with the current model & tokenizer,
        constructs a prompt for each evaluation example, generates an answer,
        and then computes BERTScore F1 between the generated answers and gold answers.

        The eval_df must contain the columns:
        - 'question'
        - 'answer'
        - 'relevant_passage_ids'

        Args:
            eval_df (pd.DataFrame): The evaluation DataFrame.
            max_eval (int): Maximum number of samples to evaluate.
            max_new_tokens (int): Maximum number of new tokens to generate beyond the prompt.
            do_sample (bool): Whether to use sampling during generation.

        Returns:
            tuple: (bert_f1, predictions, references)
                - bert_f1 (float): The average BERTScore F1 score for all evaluation samples.
                - predictions (List[str]): The list of generated answers.
                - references (List[str]): The list of ground-truth answers.
        """
        if not self.doc_dict:
            raise ValueError("Document dictionary is empty. Ensure build_doc_dict(documents_df) has been called.")

        logger.info("Evaluating generator model using BERTScore...")
        gen_pipe = pipeline("text-generation", model=self.model, tokenizer=self.tokenizer)
        # Load the BERTScore metric. (Adjust the language parameter if needed.)

        predictions = []
        references = []
        subset_df = eval_df.sample(n=min(len(eval_df), max_eval), random_state=42)

        for _, row in tqdm(subset_df.iterrows(), total=len(subset_df), desc="Evaluating"):
            question = row["question"]
            gold_answer = row["answer"]

            # Parse relevant passage IDs and combine passages.
            pid_str = str(row["relevant_passage_ids"]).strip("[]")
            doc_ids = [pid.strip() for pid in pid_str.split(",") if pid.strip()]
            relevant_texts = [self.doc_dict[pid] for pid in doc_ids if pid in self.doc_dict]
            combined_context = " ".join(relevant_texts)

            # Build a prompt that is safely truncated.
            prompt = self.build_truncated_prompt(question, combined_context, max_new_tokens)

            # Optional: verify prompt length.
            inputs = self.tokenizer(prompt, return_tensors="pt")
            input_length = inputs.input_ids.shape[1]
            if input_length > self.model.config.n_positions - max_new_tokens:
                logger.warning(f"Adjusted prompt length {input_length} exceeds allowed length; further truncation may be needed.")

            gen_output = gen_pipe(
                prompt,
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                do_sample=do_sample
            )[0]["generated_text"]

            # Extract the generated answer (everything after "Answer:")
            pred_answer = gen_output.split("Answer:")[-1].strip()
            predictions.append(pred_answer)
            references.append(gold_answer)

        gt = [[x] for x in references]
        pred = predictions
        bleu_score = corpus_bleu(gt, pred)
        logger.info(f"Bleu Score: {bleu_score:.4f}")
        return bleu_score, predictions, references

    def predict(self, test_df, max_new_tokens=32, do_sample=False):
        """
        Generate answers for a test DataFrame containing:
          - 'id', 'question', and 'relevant_passage_ids'
        The generated answers are saved in a new column "answer".
        """
        logger.info("Generating answers for test data...")
        gen_pipe = pipeline("text-generation", model=self.model, tokenizer=self.tokenizer)

        results_df = test_df.copy()
        answers = []

        for _, row in tqdm(results_df.iterrows(), total=len(results_df), desc="Generating answers"):
            question = row["question"]
            pid_str = str(row["relevant_passage_ids"]).strip("[]")
            doc_ids = [pid.strip() for pid in pid_str.split(",") if pid.strip()]
            relevant_texts = [self.doc_dict[pid] for pid in doc_ids if pid in self.doc_dict]
            combined_context = " ".join(relevant_texts)

            # Build a safe prompt via truncation.
            prompt = self.build_truncated_prompt(question, combined_context, max_new_tokens)

            inputs = self.tokenizer(prompt, return_tensors="pt")
            input_length = inputs.input_ids.shape[1]
            if input_length > self.model.config.n_positions - max_new_tokens:
                logger.warning("Prompt length exceeds allowed length after truncation.")

            gen_output = gen_pipe(
                prompt,
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                do_sample=do_sample
            )[0]["generated_text"]

            pred_answer = gen_output.split("Answer:")[-1].strip()
            answers.append(pred_answer)

        results_df["answer"] = answers
        logger.info("Answer generation complete.")
        return results_df

from transformers import pipeline
import torch

# Initialize generator with a RAM-efficient model
gen = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,  # FP16 for memory savings
    device="cuda"
)

def prepare_context(passage_ids, doc_dict):
    """Helper to build context from passage IDs"""
    return " ".join([doc_dict.get(str(pid), "") for pid in eval(passage_ids)])

# Build document dictionary (alternative to class method)
doc_dict = dict(zip(documents_df['id'].astype(str), documents_df['passage']))

def evaluate_generator(gen, eval_df, doc_dict, max_eval=5):
    references = []
    predictions = []

    for _, row in eval_df.head(max_eval).iterrows():
        context = prepare_context(row['relevant_passage_ids'], doc_dict)
        prompt = f"Answer based on context:\nContext: {context}\nQuestion: {row['question']}\nAnswer:"

        output = gen(
            prompt,
            max_new_tokens=32,
            do_sample=False
        )[0]['generated_text']

        pred = output.split("Answer:")[-1].strip()
        predictions.append(pred)
        references.append(row['answer'])

    bleu = corpus_bleu([[ref] for ref in references], predictions)
    return bleu, predictions, references

# Run evaluation
bleu_score, preds, refs = evaluate_generator(gen, train_df, doc_dict)
print(f"Initial BLEU: {bleu_score:.4f}")
for p, r in zip(preds, refs):
    print(f"\nReference: {r}\nPredicted: {p}")

#!pip install peft accelerate

from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM

# Reload model for training
model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Add LoRA
peft_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

retriever_predictions = pd.read_csv('stage_1_prediction.csv') # we load the predictions from Retriever model for 900 test questions

def clean_ticks(passage_ids):
    return [str(x).replace('\'','') for x in passage_ids.strip('[]').split(', ')]
retriever_predictions['relevant_passage_ids'] = retriever_predictions['relevant_passage_ids'].apply(clean_ticks)

# Batch processing (add this new cell)
def batch_predict(df, batch_size=4):
    results = []
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        answers = gen(
            [f"Q: {q}\nA:" for q in batch["question"]],
            max_new_tokens=64,
            do_sample=False
        )
        results.extend([a[0]['generated_text'].split("A:")[-1].strip() for a in answers])
    df["answer"] = results
    return df

result_df = batch_predict(retriever_predictions)

result_df
result_df["answer"] = result_df["answer"].fillna("").astype(str)
result_df.loc[result_df['answer']=='','answer'] = 'No answer'
result_df.to_csv('ssome_baseline.csv', index=None)

DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co


2025-04-28 18:46:02,451 - urllib3.connectionpool - DEBUG - Resetting dropped connection: huggingface.co


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/config.json HTTP/1.1" 200 0


2025-04-28 18:46:02,788 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/config.json HTTP/1.1" 200 0


DEBUG:filelock:Attempting to acquire lock 138326671828176 on /root/.cache/huggingface/hub/.locks/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/4ea05f1bc289d48ba9b92eea2f58ad8acd3dce5d.lock


...

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


2025-04-28 18:46:38,681 - accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/generation_config.json HTTP/1.1" 200 0


2025-04-28 18:46:39,907 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /TinyLlama/TinyLlama-1.1B-Chat-v1.0/resolve/main/generation_config.json HTTP/1.1" 200 0


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
